# 20190715 DAY11

## Crawling

BeautifulSoup 은 html로 parsing

scrapy는 봇

file repository / work-pool handler가 저장되는 부분



In [14]:
from bs4 import BeautifulSoup

In [17]:
import html

In [119]:
import requests

def download(method, url, params=None, data=None, headers=None):
    try:
        resp = requests.request(method, url, params=params,data=data,headers=headers)
    except requests.exceptionsp.HTTPerror as e:
        print(e.code)
        print(e.reason)
    
    return resp


In [30]:
url = "https://www.google.com/search?ei=jdgrXYnLPISr8QW3_pewBA&q=%EB%B0%95%EB%B3%B4%EC%98%81&oq=%EB%B0%95%EB%B3%B4%EC%98%81&gs_l=psy-ab.3..0l10.1850843.1852624..1852692...3.0..1.116.1082.2j9......0....1..gws-wiz.....0..0i10j0i131j0i10i42._FL24nch5gE"

html = download("get",url)

In [31]:
dom = BeautifulSoup(html.text,"lxml")
[requests.compat.urljoin(url,_["href"]) for _ in dom.find_all("a") if len(_["href"]) > 3]

['https://www.google.com/?sa=X&ved=0ahUKEwi9wvri67XjAhVjNKYKHeY_A54QOwgC',
 'https://www.google.com/?output=search&ie=UTF-8&sa=X&ved=0ahUKEwi9wvri67XjAhVjNKYKHeY_A54QPAgE',
 'https://www.google.com/search?q=%EB%B0%95%EB%B3%B4%EC%98%81&ie=UTF-8&gbv=1&sei=G-ArXf3IKuPomAXm_4zwCQ',
 'https://www.google.com/search?q=%EB%B0%95%EB%B3%B4%EC%98%81&ie=UTF-8&source=lnms&tbm=isch&sa=X&ved=0ahUKEwi9wvri67XjAhVjNKYKHeY_A54Q_AUICCgB',
 'https://www.google.com/search?q=%EB%B0%95%EB%B3%B4%EC%98%81&ie=UTF-8&source=lnms&tbm=vid&sa=X&ved=0ahUKEwi9wvri67XjAhVjNKYKHeY_A54Q_AUICSgC',
 'https://www.google.com/search?q=%EB%B0%95%EB%B3%B4%EC%98%81&ie=UTF-8&source=lnms&tbm=nws&sa=X&ved=0ahUKEwi9wvri67XjAhVjNKYKHeY_A54Q_AUICigD',
 'https://maps.google.com/maps?q=%EB%B0%95%EB%B3%B4%EC%98%81&um=1&ie=UTF-8&sa=X&ved=0ahUKEwi9wvri67XjAhVjNKYKHeY_A54Q_AUICygE',
 'https://www.google.com/search?q=%EB%B0%95%EB%B3%B4%EC%98%81&ie=UTF-8&source=lnms&tbm=shop&sa=X&ved=0ahUKEwi9wvri67XjAhVjNKYKHeY_A54Q_AUIDCgF',
 'https://www.g

In [165]:
def parseURL(seed):
    html = download("get", seed)
    dom = BeautifulSoup(html.text,"lxml")
    
    return [requests.compat.urljoin(seed, _["href"])]

    
    if len(dom.select("#mainFrame")) < 1:
        return []
    
#     seed = requests.compat.urljoin(seed,dom.select("#mainFrame")[0]["src"])

#     html = download("get",seed)
#     dom = BeautifulSoup(html.text,"lxml")
    
    return [(requests.compat.urljoin(seed[0],_["href"]),seed[1]+1) for _ in dom.find_all("a") if seed[1] < 2 and _.has_attr("href") and len(_["href"]) > 3 and checkBlog(requests.compat.urljoin(seed[0],_["href"]))]
# #으로 시작하는 것은 빼야하므로 len보다는 #으로 시작하는 것을 골라서 빼는 것이 좋다

In [27]:
import time

In [101]:
queue = list()
# queue.append("https://blog.naver.com/imagetech1?Redirect=Log&logNo=221530358950")
# seen = list()
queue.extend([(_["href"],1) for _ in dom.select("#blogColl a.f_link_b")])
while queue:
    baseURL = queue.pop(0)
    seen.append(baseURL)
    time.sleep(1)
    linkList = parseURL(baseURL)
    for link in linkList:
        if link not in queue and link not in seen:
            queue.append(link)
    print("Queue: {0}, seen: {1}".format(len(queue), len(seen)))

Queue: 3, seen: 48
Queue: 2, seen: 49
Queue: 1, seen: 50
Queue: 0, seen: 51


In [102]:
seen

['https://blog.naver.com/imagetech1?Redirect=Log&logNo=221530358950',
 'http://www.naver.com',
 'https://blog.naver.com',
 'https://blog.naver.com/prologue/PrologueList.nhn?blogId=imagetech1',
 'https://blog.naver.com/imagetech1?Redirect=Log&logNo=221530358950#blog-menu',
 'https://blog.naver.com/imagetech1?Redirect=Log&logNo=221530358950#post_1',
 'https://blog.naver.com/MyBlog.nhn',
 'https://nid.naver.com/nidlogin.login?mode=form&url=https://blog.naver.com/imagetech1',
 'https://section.blog.naver.com',
 'https://nid.naver.com/nidlogin.login?mode=form&url=https://blog.naver.com/imagetech1/221530358950',
 'https://helpx.adobe.com/kr/flash-player/kb/installation-problems-flash-player-windows.html ',
 'https://blog.naver.com/prologue/PrologueList.nhn?blogId=imagetech1&skinType=&skinId=&from=menu',
 'https://blog.naver.com/PostList.nhn?blogId=imagetech1&skinType=&skinId=&from=menu',
 'https://blog.naver.com/PostList.nhn?blogId=imagetech1&categoryNo=108&skinType=&skinId=&from=menu',
 'ht

In [100]:
url = "https://search.daum.net/search?nil_suggest=btn&w=tot&DA=SBC&q=%EC%9C%A4%EC%95%84"
# params = {"q":"박보영"}
headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"}

html = download("get",url, headers=headers)
dom = BeautifulSoup(html.text,"lxml")

In [96]:
dom.select("#blogColl a.f_link_b")[0]["href"]

'http://ghltodrodls.tistory.com/121'

In [71]:
[_["href"] for _ in dom.select("a.sh_blog_title._sp_each_url._sp_each_title") if checkBlog(_["href"])]

[]

In [48]:
queue = list()
queue.extend([_.find_parent()["href"] for _ in dom.select(".LC20lb")])
seen = list()
while queue:
    baseURL = queue.pop(0)
    seen.append(baseURL)
    time.sleep(1)
    linkList = parseURL(baseURL)
    for link in linkList:
        if link not in queue and link not in seen:
            queue.append(link)
    print("Queue: {0}, seen: {1}".format(len(queue), len(seen)))

In [63]:
requests.compat.urlparse("https://blog.naver.com/dlqlwm14?Redirect=Log&logNo=221529807195")

ParseResult(scheme='https', netloc='blog.naver.com', path='/dlqlwm14', params='', query='Redirect=Log&logNo=221529807195', fragment='')

In [106]:
def checkBlog(url):
    return requests.compat.urlparse(url)[1].endswith("tistory.com")

In [109]:
checkBlog("adam24eve.tistory.com")

False

In [110]:
table1 = id, url, seen, date
table2 = id, netloc, date
url = netloc + path + param

NameError: name 'date' is not defined

In [266]:
import sqlite3
con = sqlite3.connect("inbound1.db")
cur = con.cursor()

In [267]:
cur.executescript("""
    DROP TABLE IF EXISTS table1;
    
    CREATE TABLE table1 (
    id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    table2_id INTEGER NOT NULL,
    path TEXT NOT NULL,
    param TEXT,
    depth INTEGER NOT NULL,
    inbound INTEGER NOT NULL,
    seen BOOLEAN DEFAULT FALSE NOT NULL,
    date TIMESTAMP DEFAULT CURRENT_TIMESTAMP NOT NULL
    );
    DROP TABLE IF EXISTS table2;

    CREATE TABLE table2 (
    id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    netloc TEXT NOT NULL,
    date TIMESTAMP DEFAULT CURRENT_TIMESTAMP NOT NULL
    );
""")
# DROP TABLE IF EXISTS table1;
con.commit()

In [268]:
[_.find_parent()["href"] for _ in dom.select(".LC20lb")]

['https://namu.wiki/w/%EB%B0%95%EB%B3%B4%EC%98%81',
 'https://ko.wikipedia.org/wiki/%EB%B0%95%EB%B3%B4%EC%98%81',
 'https://ko.wikipedia.org/wiki/%EB%B0%95%EB%B3%B4%EC%98%81%EC%9D%98_%EC%9E%91%ED%92%88_%EB%AA%A9%EB%A1%9D',
 'https://news.joins.com/article/22895953',
 'https://movie.daum.net/person/main?personId=105244',
 'http://m.cafe.daum.net/parkboyoungfd',
 'https://www.msn.com/ko-kr/entertainment/tv/hd%EC%8A%A4%ED%86%A0%EB%A6%AC-%EC%9E%91%EC%9D%80-%EA%B1%B0%EC%9D%B8-%EB%B0%95%EB%B3%B4%EC%98%81%E2%80%A6%EA%B7%B8%EC%9D%98-%EC%97%B0%EA%B8%B0%EC%97%90-%EB%8C%80%ED%95%9C-%ED%9D%94%EC%A0%81%EB%93%A4/ar-BBUvL0V',
 'https://twitter.com/hashtag/%EB%B0%95%EB%B3%B4%EC%98%81',
 'http://www.epochtimes.co.kr/news/articleView.html?idxno=415565']

In [269]:
requests.compat.urlparse(html.request.url)

ParseResult(scheme='https', netloc='www.google.com', path='/search', params='', query='q=%EB%B0%95%EB%B3%B4%EC%98%81', fragment='')

In [270]:
# Seed URLs -> DB Insert
# 큐 생성~~~~

headers = {"user-agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"}
url = "https://www.google.com/search"
html = download("get", url, headers=headers, params={"q":"박보영"})
dom = BeautifulSoup(html.text, "lxml")

# url 꺼내서 파싱하기
for href in [_.find_parent()["href"] for _ in dom.select(".LC20lb")]:
    _urlparse = requests.compat.urlparse(href)
    # 베이스url
    netloc = "://".join(_urlparse[:2]) 
    # 있니?
    cur.execute("SELECT id FROM table2 WHERE netloc=? LIMIT 0,1", [netloc])
    # 있든없든 결과. 있으면 튜플 하나 없으면 none
    netlocID = cur.fetchone()
    
    # 비어있으면 아래 구문 출력
    if not netlocID:
        # 비어있으니까 table2에 넣고
        cur.execute("INSERT INTO table2(netloc) VALUES(?)", [netloc])
        # 반영하자
        con.commit()
        # 넣었으니까 아이디 값 가져올 수 있음
        cur.execute("SELECT id FROM table2 WHERE netloc=? LIMIT 0,1", [netloc])
        # 이거 튜플이니까
        netlocID = cur.fetchone()
      
    # 0이 id, 
    cur.execute("INSERT INTO table1(table2_id, path, param, depth, inbound) VALUES (?, ?, ?, ?, ?)", [netlocID[0], _urlparse[2], _urlparse[4], 1, 0])
    con.commit()

In [271]:
i = 0
while True:
    cur.execute("""
        SELECT table1.id, table2.netloc,
            table1.path, table1.param, table1.depth, table2.id
        FROM table1 
        JOIN table2 
        ON table1.table2_id=table2.id 
        WHERE table1.seen=FALSE and table1.depth < 3
        ORDER BY table1.date ASC 
        LIMIT 0,1;
    """)
    # pop 했다
    seed = cur.fetchone()
    if not seed or i>1000:
        break;
    i += 1
    
    # seen으로 넘기는건뎅 
    cur.execute("""
        UPDATE table1
        SET seen=TRUE
        WHERE id=?
    """, [seed[0]])
    con.commit()
    
    baseURL = "{0}{1}?{2}".format(seed[1], seed[2], seed[3])
    for href in parseURL(baseURL):
        _urlparse = requests.compat.urlparse(href)
        netloc = "://".join(_urlparse[:2])
        cur.execute("SELECT id FROM table2 WHERE netloc=? LIMIT 0,1", [netloc])
        netlocID = cur.fetchone()

        if not netlocID:
            cur.execute("INSERT INTO table2(netloc) VALUES(?)", [netloc])
            con.commit()
            cur.execute("SELECT id FROM table2 WHERE netloc=? LIMIT 0,1", [netloc])
            netlocID = cur.fetchone()

        cur.execute("INSERT INTO table1(table2_id, path, param, depth, inbound) VALUES (?, ?, ?, ?, ?)", [netlocID[0], _urlparse[2], _urlparse[4], seed[4]+1, seed[5]])
        con.commit()

AttributeError: module 'requests' has no attribute 'exceptionsp'

In [243]:
def parseURL(seed):
    html = download("get", seed)
    dom = BeautifulSoup(html.text, "lxml")
    
    return [requests.compat.urljoin(url, _["href"]) for _ in dom.find_all("a") if _.has_attr("href") and len(_["href"]) > 3]

In [ ]:
url = "https://www.google.com/search"
html = download("get", url, param={"q":"박보영"})
dom = BeautifulSoup(html.text,"lxml")


queue = list()
queue.append([(_.find_parent()["href"],1) for _ in dom.select(".LC20lb")])
seen = list()
queue.extend([(_["href"],1) for _ in dom.select("#blogColl a.f_link_b")])
while queue:
    baseURL = queue.pop(0)
    seen.append(baseURL)
    time.sleep(1)
    linkList = parseURL(baseURL)
    for link in linkList:
        if link not in queue and link not in seen:
            queue.append(link)
    print("Queue: {0}, seen: {1}".format(len(queue), len(seen)))

In [265]:
cur.close()
con.close()

select inbound, count(inbound)

from table1 where table2_id=2 group by inbound;

In [274]:
cur.execute("""
select inbound, count(inbound)
from table1 where table2_id=2 group by inbound;
""")
cur.fetchall()

[(0, 2), (2, 20)]

In [275]:
cur.execute("""
select count(inbound)
from table1 where inbound=1;
""")
cur.fetchall()

[(1322,)]